# Preprocessing des images .TIF du dataset lnria Aerial Image Labeling Dataset

5000x5000 px en résolution 0.3 m /px 
vers
1024x1024 px en résolution 0.6 m /px

In [1]:
import os 
from osgeo import gdal 

In [2]:
dataset_dir = r"C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset"

train_dir = os.path.join(dataset_dir, 'train', 'images')
test_dir  = os.path.join(dataset_dir, 'test', 'images')

In [11]:
def res(dataset_dir, res_dir, file_name, res=0.6):
    file_path = os.path.join(dataset_dir, file_name)
    name, ext = file_name.split('.')
    res_path = os.path.join(res_dir, name+'_s.'+ext)
    !gdalwarp -tr $res $res $file_path $res_path

In [ ]:
tmp_path = os.path.join(dataset_dir, 'tmp')
if not os.path.isdir(tmp_path):
    os.mkdir(tmp_path)

lst = os.listdir(train_dir)
for i in lst:
    if i.startswith('austin') or i.startswith('chicago') or i.startswith('vienna'):
        res(train_dir, tmp_path, i, res=0.6)
        
test_lst = os.listdir(test_dir)
for j in test_lst:
    if j.startswith('bellingham') or j.startswith('bloomington') or j.startswith('sfo'):
        res(test_dir, tmp_path, j, res=0.6)

In [ ]:
lst